# Your friend just called, she is having people over her house in Berkeley and wants you to come.
#### How do you get there? Take BART? Take Uber? Drive yourself?
<img src="images/bart_map.gif">

- This project attempted to answer the first question by predicting daily capacity of each BART station.
- **Final output here http://bart-capacity-predictions.com.s3-website-us-east-1.amazonaws.com/**
- Sample below <img src="images/sample_predictions.png">

###### Data Sources
- 1) Pull historic BART ridership information per station from http://www.bart.gov/about/reports/ridership
- 2) Pull historic weather information from https://www.wunderground.com/history/?MR=1
- 3) Ingest every 10 minutes currently arriving BART trains from http://api.bart.gov/docs/overview/index.aspx
- 4) Ingest every 10 minutes the current weather in San Francisco http://openweathermap.org/api

###### Analysis Procedure
- 1) Pull live BART data into S3 (which is acting as a data lake) using kinesis to manage load and scale.
<img src="images/kinesis_overview.png">
- 2) Upload the historic data to s3
- 3) Use Spark on EMR to normalize the data to third normal form and push back to s3
<img src="images/emr_spark.jpg">
- 4) Output the normalized data to mongoDB
- 5) Create a gradient boost model in Spark from the daily normalized data to predict station ridership
- 6) Output the prediction to a s3 bucket acting as a website


#### Overview of scalability and robustness of the system
Real-Time BART Arrival, Capacity, and Predicted Ridership


> *Robustness and fault tolerance*
- Spark is very fault tolerant thanks to the directed aclyic graph structure (https://www.sigmoid.com/apache-spark-internals/) that it employes to manage nodes that go down during computation. This graph keeps an outline of each transformation made to the underlying dataset. Therefore, these transformations can be re-created if need be. 
- There is separation of concerns in this system by having different ec2 instances. This setup ensures that there is not a single point of failure in case one server goes down, the rest of the system will not also go down with it. 
- For version 2.0, additional ec2 instance can be spun up for the webserver in case our server can't handle excess web traffic.

> *Low latency reads and updates*
- Spark has very low latency
- Amazon kinesis can have have low as a recommended one second streams written to s3. http://docs.aws.amazon.com/streams/latest/dev/kinesis-low-latency.html
- For the next version, Spark Streaming can be implemented which can have latency as low as one millisecond.


> *Scalability*
- Amazon kinesis is great for data scalability as it can handle alarge inflow of data that varies over time. The way to set up kinesis is to specify two parameters 
    - 1) How much time do you want to allow inbetween writes to s3
    - 2) How large do you want to file to be that you are writting to s3
- After these parameters are set, whenever one of the thresholds is hit, then a new file will be written to s3. 
- Kinesis is also very useful for maintaining a file directory in s3 as it automatically creates year, month, day, and hour folder systems. 
- Checks such as
autoscaling (http://docs.aws.amazon.com/autoscaling/latest/userguide/WhatIsAutoScaling.html)
can ensure that the website hosting the EC2 instance does not go down. Or, when
additional traffic comes to the website, additional servers can spin up
to handle the load. I currently have alarms set up and autoscaling will be implemented in version 2.0.
- In addition, for future versions I can add a load balancer to my EC2 instances to handle we

> *Generalization*
- Right now, this system has been optimized to predict ridership per station per day.
- However, with the data that we have stored, the following questions can be asked and implemented fairly easily.
    - Predict the weather given the ridership yesterday per station along with the day of the month and the month number
    - Which stations on average are the most crowded?
    - How many train stops are made per day through BART?
- Version 2.0 will aim for a web app that has a querable interface to interact directly with Mongodb.


> *Extensibility*
- This system can be used across any EC2/EMR instance assuming that all dependencies have been installed.
- All of the code used is localized here on github and can be deployed to any new server. 
- In the future, a vagrant file can be configure to keep the dependencies of the system. This way, when a new EC2 isntance is spun up, all of the libraries needed can be automatically installed as well. (https://github.com/mitchellh/vagrant-aws)
- Alongside this, the different severs all for development to be run on one part of the system without disrupting th other parts of the system.

> *Ad hoc queries*
- With all of the data being stored in Mongodb, a user can write custom queries
to ask different questions of historical data than is being asked currently. In version 2.0, this database will be hosted using a flask app to allow queries from the web versus logging into a ubuntu server. (http://blog.dwyer.co.za/2013/10/a-basic-web-app-using-flask-and-mongodb.html)

> *Minimal Maintenance*
- With airflow, emails can be sent in case of errors in completing tasks.
- In addition, Amazon allows you to set up alerts in case your server experiences any errors.
<img src="images/mongo_db_error.png">


> *Debuggability*
- Data structures will are immutable in this system. Therefore, it will be easier
to debug what things go wrong in Spark. This allows a user to investigate each intermediate data structure to understand what transformations were made on the data.
- Airflow keeps all log files from the tasks run in a centralized location which makes it easier to parse through.
- By separating each task into a different server, it is easier to debug when a part of the system crashes. 
- Kinesis comes with built in Amazon Cloudwatch support which will send me emails to ensure that my kinesis stream is running. <img src="images/cloudwatch_kinesis.png">


### Below is an overview of the architecture, as well as some technologies used in this project

# Architecture of the system
<img src="images/data_architecture.png">
> Here is the overview of this system


# Overview of airflow
## Airflow is a great tool to replace cron jobs. Instead of hoping the your cron jobs have worked, you have a great UI and task dependencies to understand how you system is running.
- The main page below shows what tasks have succeeded, are running, and failed. For tasks that have failed, airflow will send you an email outlining the error encountered.
- In addition, the airflow UI contains all of your log files so that you can understand what is happening within your tasks.
> A quick overview of how to get set up to run Airflow.
    - Create your DAG script following the tutorial at https://airflow.incubator.apache.org/tutorial.html.
        - Make sure to have the **start date** be yesterday. Otherwise, the DAG will run for the days it misssed (i.e. if today is 3/7/2017 and your start date in your dag is 3/1/2017, you will have seven dags if you are running a daily job.
    - Place the dag.py script in your home directory
    - Run the dag script with python your_dag_script.py
    - This should have created a new folder in your home directory called `Airflow`
    - cd to `Airflow` and mkdir dags
    - Move your_dag_scipt.py to the dags foler
    - re-run your dag script (python your_dag_script.py)
    - Launch the Airflow db (airflow initdb)
    - Launch the Airflow scheduler
    - Launch the Airflow webserver 
    - Additional details can be found here. http://site.clairvoyantsoft.com/installing-and-configuring-apache-airflow/
<img src="images/airflow_overview.png">

#### The dependencies screen below show you the directed acyclic graph of your tasks.
<img src="images/airflow_dependencies.png">


# Schema in Third Normalized Form
- For this project, there are four data sources
    - 1) 2016 BART historic ridership information
    - 2) 2016 San Francisco (94105) weather information
    - 3) Live BART train arrivals
    - 4) Live weather predictions
- Below, is how I normalized this data.
<img src="images/final_schema.png">
